БИБЛИОТЕКИ

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [2]:
# монтируем свой диск...
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


ФАЙЛЫ ВХОДНЫХ ДАННЫХ

In [7]:
# файл с тестовой выборкой
x_test_file = '/content/drive/My Drive/Bases/DTP/x_test.dat'

# исходный файл с тестовой таблицей
test_dataset_file = '/content/drive/My Drive/Bases/DTP/test_dataset_test.csv'

# веса нулевой модели
model_0_file = '/content/drive/My Drive/Bases/DTP/043_1/model_0.h5'

# веса первой модели
model_1_file = '/content/drive/My Drive/Bases/DTP/043_1/model_1.h5'

# веса второй модели
model_2_file = '/content/drive/My Drive/Bases/DTP/043_1/model_2.h5'

# файл результата
result_dataset_file = '/content/drive/My Drive/Bases/DTP/res_dataset.csv'


ЧТЕНИЕ ТЕСТОВОЙ ВЫБОРКИ

In [4]:
test_size = 6788

x_test = np.fromfile(x_test_file)
x_test = x_test.reshape((test_size, 7))

МОДЕЛИ НЕЙРОННОЙ СЕТИ

In [5]:
# Для Шагов 1 и 2
def CreateModel_binary(activation='linear', dropout=0.1):

  input = Input((7,)) 

  x = BatchNormalization()(input)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)
  
  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  output = Dense(1, activation='sigmoid')(x) 
  
  return Model(input, output) 

# Для Шага 3
def CreateModel_regression(activation='linear', dropout=0.1):

  input = Input((7,)) 

  x = BatchNormalization()(input)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)
  
  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  x = BatchNormalization()(x)
  x = Dense(200, activation=activation)(x)
  x = Dropout(dropout)(x)

  output_1 = Dense(1, activation='relu')(x) 
  output_2 = Dense(1, activation='relu')(x) 
  output_3 = Dense(1, activation='relu')(x) 
  output_4 = Dense(1, activation='relu')(x) 

  return Model(input, [output_1, output_2, output_3, output_4]) 

ПРОПУСК ЧЕРЕЗ МОДЕЛЬ РЕГРЕССИЮ ШАГ 3

In [8]:
model = CreateModel_regression('relu')

# чтение весов
model.load_weights(model_2_file)

y_pred = model.predict(x_test)

y_0 = np.around(y_pred[0]).astype(int)
y_1 = np.around(y_pred[1]).astype(int)
y_2 = np.around(y_pred[2]).astype(int)
y_3 = np.around(y_pred[3]).astype(int)

y_0 = y_0.reshape((test_size, ))
y_1 = y_1.reshape((test_size, ))
y_2 = y_2.reshape((test_size, ))
y_3 = y_3.reshape((test_size, ))

# результаты в списки
y_d = list(y_0)
y_dc = list(y_1)
y_w = list(y_2)
y_wc = list(y_3)

del model

ТЕПЕРЬ ОТСЕКАЕМ КЛАССЫ '0000' и '0010'

In [9]:
# нулевой список маркеров (0 - строка не задействована в классе, 1 - задействована)
y_markers = [0 for i in range(len(y_d))]

# создание модели бинарной классификации
model = CreateModel_binary('relu')

ШАГ 0. Класс 0000

In [10]:
# чтение весов
model.load_weights(model_0_file)

# предсказание
y_pred = np.around(model.predict(x_test)).astype(int)  
y_pred = y_pred[:, 0] 

# обработка предсказания
for i in range(y_pred.shape[0]):
  if (y_markers[i] == 0) and (y_pred[i] == 0):
    y_markers[i] = 1
    y_d[i] = 0
    y_dc[i] = 0
    y_w[i] = 0
    y_wc[i] = 0

ШАГ 1. Класс 0010

In [11]:
# чтение весов
model.load_weights(model_1_file)

# предсказание
y_pred = np.around(model.predict(x_test)).astype(int)  
y_pred = y_pred[:, 0] 

# обработка предсказания
for i in range(y_pred.shape[0]):
  if (y_markers[i] == 0) and (y_pred[i] == 0):
    y_markers[i] = 1
    y_d[i] = 0
    y_dc[i] = 0
    y_w[i] = 1
    y_wc[i] = 0

ЗАВЕРШЕНИЕ

In [12]:
# загружаем тестовую базу данных...
df = pd.read_csv(test_dataset_file)

# читаем id в список
id = list(df['id'])

# создаем новую таблицу из id и предсказанных данных
data = {'id': id, 'Погибло': y_d, 'Погибло детей': y_dc, 
        'Ранено': y_w, 'Ранено детей': y_wc}
df = pd.DataFrame(data)

# сохраняем результат
df.to_csv(result_dataset_file, index=False)

print(df)

             id  Погибло  Погибло детей  Ранено  Ранено детей
0     490078911        0              0       0             0
1     490055448        0              0       0             0
2     490054440        0              0       0             0
3     490037847        0              0       0             0
4     490037049        0              0       0             0
...         ...      ...            ...     ...           ...
6783  490014641        0              0       3             1
6784  490033115        0              0       3             1
6785  490008221        0              0       3             1
6786  490119585        0              0       0             0
6787  490109680        0              0       0             0

[6788 rows x 5 columns]
